[20220924]

# 2. Model 6

After Preparation ready. Build and train the model in here. its using LSTM one layer. with data gen4

## Goal
Find patterns on the features for target

## Plan
Checklist what will be done on the notebook :

        [ ] Get Data
        [ ] Modelling
        [ ] Evaluation

In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
from statsmodels.tsa.arima.model import ARIMA
import sys
sys.path.insert(1,'../../src/')

## import myfunc inside src folder
from ramal import eval_model, jampi

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import metrics, optimizers
import keras
import tensorflow as tf
from keras.models import load_model

from sklearn.model_selection import train_test_split

2022-09-28 16:32:36.265136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 16:32:36.265188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/local/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
with open("../../config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

PATH_RAW = config['path']['raw']
PATH_INTERIM = config['path']['interim']
PATH_PROCESSED = config['path']['processed']
# PATH_MODEL = config['path']['artifact']+os.getcwd().split('\\')[-1]+'/'
PATH_MODEL = config['path']['artifact']+'iteration_3'+'/'
PATH_UDF = config['path']['udf']

## Get Data

In [4]:
Xt, yt, Xo, yo = joblib.load(PATH_INTERIM+'feat_target_gen4_iter3.pkl')
Xt = Xt.astype("float32")
yt = yt.astype("float32")

Xo = Xo.astype("float32")
yo = yo.astype("float32")

In [5]:
Xt.shape

(409713, 15, 2)

In [6]:
yt.shape

(409713, 1)

In [7]:
SEQ = Xt.shape[1]

In [8]:
## not the best way to split time series dataset
x_tr, x_test, y_tr, y_test = train_test_split(
    Xt, yt, test_size=0.25, random_state=42)

x_train, x_val, y_train, y_val = train_test_split(
    x_tr, y_tr, test_size=0.2, random_state=42)

## Model 1

In [9]:
# os.environ["CUDA_VISIBLE_DEVICES"] 

In [10]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


2022-09-28 16:32:50.395805: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-28 16:32:50.395899: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-28 16:32:50.395923: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (62cd00eaf0b9): /proc/driver/nvidia/version does not exist


In [11]:
mod_ts = Sequential()
mod_ts.add(LSTM(128, input_shape=(SEQ, 2), return_sequences=True))
mod_ts.add(LSTM(64))
mod_ts.add(Dense(32))
mod_ts.add(Dense(16))
mod_ts.add(Dense(1))


2022-09-28 16:33:36.593579: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',mode='min', patience=6) 

check_p = keras.callbacks.ModelCheckpoint(
        filepath=PATH_MODEL+'ws_data_model6_iter3.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True)

opt = tf.keras.optimizers.Adam(learning_rate=0.00035)

mod_ts.compile(loss='mean_squared_error', optimizer=opt, metrics=[
        tf.keras.metrics.RootMeanSquaredError(name='the_rmse'),
        tf.keras.metrics.MeanAbsolutePercentageError(name='the_mape'),
    ])

In [13]:
history = mod_ts.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=50,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
    callbacks=[early_stop,check_p]
)

Epoch 1/50
1921/1921 [==============================] - 99s 50ms/step - loss: 0.7110 - the_rmse: 0.8432 - the_mape: 135926.8906 - val_loss: 0.6622 - val_the_rmse: 0.8138 - val_the_mape: 142700.3125
Epoch 2/50
1921/1921 [==============================] - 97s 50ms/step - loss: 0.6369 - the_rmse: 0.7981 - the_mape: 122603.1953 - val_loss: 0.6035 - val_the_rmse: 0.7768 - val_the_mape: 132301.1250
Epoch 3/50
1921/1921 [==============================] - 100s 52ms/step - loss: 0.5975 - the_rmse: 0.7730 - the_mape: 129068.0469 - val_loss: 0.5890 - val_the_rmse: 0.7674 - val_the_mape: 225080.0000
Epoch 4/50
1921/1921 [==============================] - 96s 50ms/step - loss: 0.5719 - the_rmse: 0.7563 - the_mape: 117947.1953 - val_loss: 0.5546 - val_the_rmse: 0.7447 - val_the_mape: 163074.9688
Epoch 5/50
1921/1921 [==============================] - 96s 50ms/step - loss: 0.5534 - the_rmse: 0.7439 - the_mape: 119257.1094 - val_loss: 0.5473 - val_the_rmse: 0.7398 - val_the_mape: 212634.4531
Epoch 6/5

KeyboardInterrupt: 

## Eval

In [14]:
df_epoch_hist = pd.DataFrame(history.history)
df_epoch_hist[['loss','val_loss']].plot()

NameError: name 'history' is not defined

In [15]:
# depd = {'auc_1' : metrics.AUC}
model_load = load_model(PATH_MODEL + 'ws_data_model6_iter3.h5',compile=False)

In [16]:
model_load.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 128)           67072     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 119,105
Trainable params: 119,105
Non-trainable params: 0
_________________________________________________________________


In [17]:
model_load.compile(loss='mean_squared_error', optimizer=opt, metrics=[
        tf.keras.metrics.RootMeanSquaredError(name='the_rmse'),
        tf.keras.metrics.MeanAbsolutePercentageError(name='the_mape'),
    ])

In [18]:
model_load.evaluate(x_test,y_test)

3201/3201 [==============================] - 31s 9ms/step - loss: 0.4812 - the_rmse: 0.6937 - the_mape: 154794.1094


[0.48118048906326294, 0.6936717629432678, 154794.109375]

In [19]:
model_load.evaluate(Xo,yo)

521/521 [==============================] - 5s 9ms/step - loss: 0.4654 - the_rmse: 0.6822 - the_mape: 131929.4844


[0.46537989377975464, 0.682187557220459, 131929.484375]